In [19]:
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import numpy as np

In [20]:
user_data = pd.read_csv('Data/UserDummyData.csv', delimiter=';')
food_data = pd.read_csv('Data/FoodDummyData.csv', delimiter=';')

In [21]:
user_data_check = user_data.iloc[:, 11:]
count_non_zeros = user_data_check.astype(bool).sum(axis=1)

user_data_dup = pd.DataFrame()
columns_to_duplicate = user_data.columns[:11]

for i, row in user_data.iterrows():
    user_data_dup = user_data_dup.append([row[columns_to_duplicate]]*count_non_zeros[i])

user_data_dup = user_data_dup.reset_index(drop=True)

In [22]:
food_data_dup = pd.DataFrame()
rating = []

for i, row in user_data_check.iterrows():
    for j, item in enumerate(row):
        if item != 0:
            column_name = user_data_check.columns[j]
            rating.append(item)
            food_row = food_data[food_data['Nama'] == column_name]
            
            if not food_row.empty:
                food_data_dup = food_data_dup.append(food_row)

food_data_dup = food_data_dup.reset_index(drop=True)

In [23]:
train_user_data = user_data_dup.drop(columns=['ID', 'Nama'])
train_food_data = food_data_dup.drop(columns=['ID', 'Nama'])

In [24]:
x = train_food_data.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
train_food_data = pd.DataFrame(x_scaled, columns=train_food_data.columns)

In [25]:
x = train_user_data.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
train_user_data = pd.DataFrame(x_scaled, columns=train_user_data.columns)

In [26]:
train_user_data = np.array(train_user_data)
train_food_data = np.array(train_food_data)

In [27]:
rating = np.array(rating)

In [31]:
train_food_data.shape

(4203, 13)

In [32]:
num_outputs = 16
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_outputs)
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_outputs)
])

input_user = tf.keras.layers.Input(shape=(9))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

input_item = tf.keras.layers.Input(shape=(13))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)

In [33]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 13)]         0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 16)           848         ['input_3[0][0]']                
                                                                                                  
 sequential_3 (Sequential)      (None, 16)           976         ['input_4[0][0]']                
                                                                                            

In [36]:
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt,
              loss=cost_fn)

In [38]:
model.fit([train_user_data, train_food_data], rating, epochs=30)

Epoch 1/30
132/132 [==============================] - 1s 6ms/step - loss: 5.9400
Epoch 2/30
132/132 [==============================] - 1s 5ms/step - loss: 5.9400
Epoch 3/30
132/132 [==============================] - 1s 5ms/step - loss: 5.9400
Epoch 4/30
132/132 [==============================] - 1s 5ms/step - loss: 5.9400
Epoch 5/30
132/132 [==============================] - 1s 5ms/step - loss: 5.9400
Epoch 6/30
132/132 [==============================] - 1s 6ms/step - loss: 5.9400
Epoch 7/30
132/132 [==============================] - 1s 5ms/step - loss: 5.9400
Epoch 8/30
132/132 [==============================] - 1s 5ms/step - loss: 5.9400
Epoch 9/30
132/132 [==============================] - 1s 6ms/step - loss: 5.9400
Epoch 10/30
132/132 [==============================] - 1s 6ms/step - loss: 5.9400
Epoch 11/30
132/132 [==============================] - 1s 6ms/step - loss: 5.9400
Epoch 12/30
132/132 [==============================] - 1s 6ms/step - loss: 5.9400
Epoch 13/30
132/132 [====